In [ ]:
filter_by_fqn: str = ""
filter_by_method_name: str = ""

Bring in data table and do initial cleaning

In [ ]:
from code_data_science import data_table as dt

df = dt.read_csv("../samples/call_graph.csv")

# drop all columns except fromClass, fromName, toClass, toName
df = df[["fromClass", "fromName", "toClass", "toName"]]

if filter_by_fqn:
    df = df[(df["fromClass"] == filter_by_fqn) | (df["toClass"] == filter_by_fqn)]

if filter_by_method_name:
    df = df[
        (df["fromName"] == filter_by_method_name)
        | (df["toName"] == filter_by_method_name)
    ]

if df.empty:
    raise Exception("DataFrame is empty")

In [ ]:
plantuml_code = ["@startuml"]
# TODO: add dynamic coloring based on actions or something in the future
colors = ["black", "red", "blue", "purple"]

classes = {}


def is_valid_class(class_name):
    if filter_by_fqn:
        return (
            True
            if isinstance(class_name, str) and class_name == filter_by_fqn
            else False
        )

    return True


def is_valid_method(method_name):
    if filter_by_method_name:
        return (
            True
            if isinstance(method_name, str) and method_name == filter_by_method_name
            else False
        )

    return True


def make_method(name):
    # return f"{name}(arguments)" if isinstance(arguments, str) else f"{name}()"
    # overloads are not supported in v1.
    return f"{name}()"


def add_method_to_class(class_name, method_name):
    if class_name not in classes:
        classes[class_name] = set()
    members = classes[class_name]
    method = make_method(method_name)
    if method not in members:
        members.add(method)


for i, row in df.iterrows():
    from_class, to_class = row["fromClass"], row["toClass"]
    from_name, to_name = row["fromName"], row["toName"]

    if is_valid_class(from_class) and is_valid_method(from_name):
        add_method_to_class(from_class, from_name)
        add_method_to_class(to_class, to_name)

        from_method = make_method(from_name)
        to_method = make_method(to_name)
        plantuml_code.append(
            f'"{from_class}::{from_method}" --> "{to_class}::{to_method}"'
        )

    if is_valid_class(to_class) and is_valid_method(to_name):
        add_method_to_class(to_class, to_name)
        add_method_to_class(from_class, from_name)

        from_method = make_method(from_name)
        to_method = make_method(to_name)
        plantuml_code.append(
            f'"{to_class}::{to_method}" <-- "{from_class}::{from_method}"'
        )

In [ ]:
# Create classes
for _class in classes:
    plantuml_code.append(f'class "{_class}"')
    plantuml_code.append("{")
    for member in classes[_class]:
        plantuml_code.append(f"    +{member}")
    plantuml_code.append("}")

# empty line needed for whatever reason plantuml needs it for when it needs it ¯\_(ツ)_/¯
plantuml_code.append("")

plantuml_code.append("@enduml")

plantuml_code_string = "\n".join(plantuml_code)

# print(plantuml_code_string)

Run the plantUML code and display the image. \
Here we are using the `-pipe` option to avoid creating temporary files.

In [ ]:
# TODO: refactor this into code-data-science
from subprocess import run
from IPython.display import SVG

jarpath = "./resources/plantuml.jar"
cmd = ["java", "-jar", jarpath, "-tsvg", "-pipe"]
compl = run(cmd, input=plantuml_code_string, text=True, capture_output=True)
SVG(compl.stdout)